In [5]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np


In [12]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm import tqdm
import pandas as pd
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

In [ ]:

# Load dataset
dataset = load_dataset("ucberkeley-dlab/measuring-hate-speech")
df = dataset['train'].to_pandas()[['text', 'violence', 'genocide', 'attack_defend', 'hatespeech']]

df['violence'] = df['violence'].astype(int)
df['genocide'] = df['genocide'].astype(int)
df['attack_defend'] = df['attack_defend'].astype(int)
df['hatespeech'] = df['hatespeech'].astype(int)

print("Violence distribution:\n", df['violence'].value_counts().sort_index())
print("Genocide distribution:\n", df['genocide'].value_counts().sort_index())
print("Attack/Defend distribution:\n", df['attack_defend'].value_counts().sort_index())
print("Hatespeech distribution:\n", df['hatespeech'].value_counts().sort_index())

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [ ]:

class HateSpeechMultiLabelDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.texts = dataframe['text'].values
        self.labels = dataframe[['violence', 'genocide', 'attack_defend', 'hatespeech']].values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        labels = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }


In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = HateSpeechMultiLabelDataset(train_df, tokenizer)
val_dataset = HateSpeechMultiLabelDataset(val_df, tokenizer)
test_dataset = HateSpeechMultiLabelDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:

class BertMultiLabel(torch.nn.Module):
    def __init__(self, num_classes=5, num_labels=4):
        super(BertMultiLabel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes * num_labels)
        self.num_classes = num_classes
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token
        logits = self.classifier(pooled_output)
        logits = logits.view(-1, self.num_labels, self.num_classes)  # Shape: (batch_size, 4, 5)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = sum(loss_fct(logits[:, i, :], labels[:, i]) for i in range(self.num_labels))

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertMultiLabel(num_classes=5, num_labels=4)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
# Training functions
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    all_predictions, all_true_labels = [], []

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        logits = outputs['logits']

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=2).cpu().numpy()  # Shape: (batch_size, 4)
        all_predictions.append(preds)
        all_true_labels.append(labels.cpu().numpy())

    all_predictions = np.vstack(all_predictions)
    all_true_labels = np.vstack(all_true_labels)
    
    avg_loss = total_loss / len(data_loader)
    accuracies = [accuracy_score(all_true_labels[:, i], all_predictions[:, i]) for i in range(4)]
    f1_scores = [f1_score(all_true_labels[:, i], all_predictions[:, i], average='macro') for i in range(4)]
    return avg_loss, accuracies, f1_scores


In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    all_predictions, all_true_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']
            logits = outputs['logits']

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=2).cpu().numpy()
            all_predictions.append(preds)
            all_true_labels.append(labels.cpu().numpy())

    all_predictions = np.vstack(all_predictions)
    all_true_labels = np.vstack(all_true_labels)

    avg_loss = total_loss / len(data_loader)
    accuracies = [accuracy_score(all_true_labels[:, i], all_predictions[:, i]) for i in range(4)]
    f1_scores = [f1_score(all_true_labels[:, i], all_predictions[:, i], average='macro') for i in range(4)]
    return avg_loss, accuracies, f1_scores


In [ ]:

num_epochs = 3
best_val_f1 = 0
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
   
    train_loss, train_accs, train_f1s = train_epoch(model, train_loader, optimizer, device)
    print(f'Train Loss: {train_loss:.4f}')
    for i, label in enumerate(['Violence', 'Genocide', 'Attack/Defend', 'Hatespeech']):
        print(f'{label} - Accuracy: {train_accs[i]:.4f}, Macro F1: {train_f1s[i]:.4f}')
    
    val_loss, val_accs, val_f1s = evaluate(model, val_loader, device)
    print(f'Validation Loss: {val_loss:.4f}')
    for i, label in enumerate(['Violence', 'Genocide', 'Attack/Defend', 'Hatespeech']):
        print(f'{label} - Accuracy: {val_accs[i]:.4f}, Macro F1: {val_f1s[i]:.4f}')
    
    avg_val_f1 = np.mean(val_f1s)
    if avg_val_f1 > best_val_f1:
        best_val_f1 = avg_val_f1
        torch.save(model.state_dict(), 'best_bert_multilabel.pt')


In [ ]:

model.load_state_dict(torch.load('best_bert_multilabel.pt'))
test_loss, test_accs, test_f1s = evaluate(model, test_loader, device)
print(f'Test Loss: {test_loss:.4f}')
for i, label in enumerate(['Violence', 'Genocide', 'Attack/Defend', 'Hatespeech']):
    print(f'{label} - Accuracy: {test_accs[i]:.4f}, Macro F1: {test_f1s[i]:.4f}')

model.eval()
all_predictions, all_true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs['logits'], dim=2).cpu().numpy()
        all_predictions.append(preds)
        all_true_labels.append(labels.cpu().numpy())

all_predictions = np.vstack(all_predictions)
all_true_labels = np.vstack(all_true_labels)


In [ ]:

print("\nPer-class metrics:")
label_info = [
    ('Violence', 5),
    ('Genocide', 5),
    ('Attack/Defend', 5),
    ('Hatespeech', 3)
]

for i, (label, num_classes) in enumerate(label_info):
    print(f"\n{label}:")
    target_names = [f'{label} {j}' for j in range(num_classes)]
    print(classification_report(all_true_labels[:, i], all_predictions[:, i], target_names=target_names))


In [21]:

def predict(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs['logits']
        preds = torch.argmax(logits, dim=2).cpu().numpy()[0]
    
    return {
        'violence': preds[0],
        'genocide': preds[1],
        'attack_defend': preds[2],
        'hatespeech': preds[3]
    }

# Example inference
texts = [
    "This is a violent threat!",
    "I love peaceful discussions.",
    "You deserve to be hurt for this."
]
for text in texts:
    pred = predict(text, model, tokenizer, device)
    print(f'Text: {text}')
    print(f'Predicted Labels: {pred}\n')

# Save model
torch.save(model.state_dict(), './fine_tuned_bert_multilabel.pt')
tokenizer.save_pretrained('./fine_tuned_bert_multilabel')

Violence distribution:
 violence
0    67922
1    30727
2    12241
3    11262
4    13404
Name: count, dtype: int64
Genocide distribution:
 genocide
0    90058
1    22838
2     8107
3     5301
4     9252
Name: count, dtype: int64
Attack/Defend distribution:
 attack_defend
0     7958
1    11046
2    38201
3    44883
4    33468
Name: count, dtype: int64
Hatespeech distribution:
 hatespeech
0    80624
1     8911
2    46021
Name: count, dtype: int64


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 1/3
Train Loss: 3.4155
Violence - Accuracy: 0.5850, Macro F1: 0.3871
Genocide - Accuracy: 0.7140, Macro F1: 0.3350
Attack/Defend - Accuracy: 0.5520, Macro F1: 0.4381
Hatespeech - Accuracy: 0.7843, Macro F1: 0.3190
Validation Loss: 3.3753
Violence - Accuracy: 0.5846, Macro F1: 0.4055
Genocide - Accuracy: 0.7139, Macro F1: 0.3284
Attack/Defend - Accuracy: 0.5614, Macro F1: 0.4506
Hatespeech - Accuracy: 0.7887, Macro F1: 0.5350
Epoch 2/3
Train Loss: 3.2215
Violence - Accuracy: 0.6008, Macro F1: 0.4183
Genocide - Accuracy: 0.7196, Macro F1: 0.3541
Attack/Defend - Accuracy: 0.5829, Macro F1: 0.4818
Hatespeech - Accuracy: 0.7984, Macro F1: 0.5427
Validation Loss: 3.3599
Violence - Accuracy: 0.5889, Macro F1: 0.3925
Genocide - Accuracy: 0.7147, Macro F1: 0.3485
Attack/Defend - Accuracy: 0.5689, Macro F1: 0.4810
Hatespeech - Accuracy: 0.7901, Macro F1: 0.5360
Epoch 3/3
Train Loss: 3.1025
Violence - Accuracy: 0.6102, Macro F1: 0.4400
Genocide - Accuracy: 0.7223, Macro F1: 0.3731
Attack/De

/tmp/ipykernel_31/1875311655.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_bert_multilabel.pt'))


Test Loss: 3.3663
Violence - Accuracy: 0.5872, Macro F1: 0.4056
Genocide - Accuracy: 0.7125, Macro F1: 0.3550
Attack/Defend - Accuracy: 0.5669, Macro F1: 0.4794
Hatespeech - Accuracy: 0.7906, Macro F1: 0.5362

Per-class metrics:

Violence:
              precision    recall  f1-score   support

  Violence 0       0.65      0.88      0.75      6763
  Violence 1       0.38      0.18      0.25      3200
  Violence 2       0.14      0.01      0.01      1187
  Violence 3       0.30      0.36      0.33      1047
  Violence 4       0.65      0.75      0.69      1359

    accuracy                           0.59     13556
   macro avg       0.42      0.44      0.41     13556
weighted avg       0.52      0.59      0.53     13556


Genocide:
              precision    recall  f1-score   support

  Genocide 0       0.75      0.97      0.85      8987
  Genocide 1       0.31      0.05      0.08      2361
  Genocide 2       0.18      0.03      0.06       790
  Genocide 3       0.22      0.07      0.10

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('./fine_tuned_bert_multilabel/tokenizer_config.json',
 './fine_tuned_bert_multilabel/special_tokens_map.json',
 './fine_tuned_bert_multilabel/vocab.txt',
 './fine_tuned_bert_multilabel/added_tokens.json')

In [27]:
!pip install gdown


In [28]:
# Save the model locally
torch.save(model.state_dict(), 'fine_tuned_bert_multilabel.pt')
tokenizer.save_pretrained('fine_tuned_bert_multilabel')


('fine_tuned_bert_multilabel/tokenizer_config.json',
 'fine_tuned_bert_multilabel/special_tokens_map.json',
 'fine_tuned_bert_multilabel/vocab.txt',
 'fine_tuned_bert_multilabel/added_tokens.json')